In [162]:
from tdc.single_pred import Epitope
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score, precision_recall_curve
from sklearn.metrics import roc_auc_score, f1_score, average_precision_score, precision_score, recall_score, accuracy_score
from copy import deepcopy
torch.manual_seed(1)
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt  
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.metrics import roc_auc_score 
from tdc.multi_pred import AntibodyAff
from collections import defaultdict
from tqdm import tqdm
from torch.distributions import Categorical, Normal

In [75]:
# data = AntibodyAff(name = 'Protein_SAbDab')
# split = data.get_split()

In [76]:
data =  Epitope(name = 'IEDB_Jespersen')
split = data.get_split()

Found local copy...
Loading...
Done!


In [77]:
train_data = split['train']
valid_data = split['valid']
test_data = split['test']

In [78]:
# def to_protlists(data):

#     protlists = list()
#     temp_dict = data.to_dict("records")
#     for record in temp_dict:
#         protlist = defaultdict(list)
#         antigen = record['Antigen_ID']
#         seq = record['Antigen']
#         desc = ''
#         protlist[antigen].append(desc)
#         protlist[antigen].append(seq)

#         protlists.append(protlist)
        
#     return protlists

In [79]:
lst = train_data['Antigen'].tolist()
maxlen = max([len(A) for A in lst])
maxlen

34350

In [81]:
X = 'Antigen'

In [82]:
def data2vocab(data):
	length = len(data)
	vocab_set = set()
	total_length, positive_num = 0, 0
	for i in range(length):
		antigen = data[X][i]
		vocab_set = vocab_set.union(set(antigen))
		Y = data['Y'][i]
		assert len(antigen) > max(Y)
		total_length += len(antigen)
		positive_num += len(Y)
	return vocab_set, positive_num / total_length

In [83]:
train_vocab, train_positive_ratio = data2vocab(train_data)
valid_vocab, valid_positive_ratio = data2vocab(valid_data)
test_vocab, test_positive_ratio = data2vocab(test_data)

In [84]:
vocab_set = train_vocab.union(valid_vocab)
vocab_set = vocab_set.union(test_vocab)
vocab_lst = list(vocab_set)
# logger

In [85]:
def onehot(idx, length):
	lst = [0 for i in range(length)]
	lst[idx] = 1
	return lst 

def zerohot(length):
	return [0 for i in range(length)]

# what is the maxlength here
def standardize_data(data, vocab_lst, maxlength = 300):
	length = len(data)
	standard_data = []
	for i in range(length):
		antigen = data[X][i]
		Y = data['Y'][i] 
		sequence = [onehot(vocab_lst.index(s), len(vocab_lst)) for s in antigen] 
		labels = [0 for i in range(len(antigen))]
		mask = [True for i in range(len(labels))] # labels and mask have the same length
		sequence += (maxlength-len(sequence)) * [zerohot(len(vocab_lst))] #pad to consistent length
		labels += (maxlength-len(labels)) * [0] 
		mask += (maxlength-len(mask)) * [False] # pad to maxlength
		for y in Y:
			labels[y] = 1 		
		sequence, labels, mask = sequence[:maxlength], labels[:maxlength], mask[:maxlength]
		sequence, labels, mask = torch.FloatTensor(sequence), torch.FloatTensor(labels), torch.BoolTensor(mask) 
		# print(sequence.shape, labels.shape, mask.shape)
        # sequence is 2D, labels and mask are 1D
		standard_data.append((sequence, labels, mask))
	return standard_data 

In [86]:
# def standardize_data2(data, vocab_lst):
# 	length = len(data)
# 	standard_data = []
# 	for i in range(length):
# 		antigen = data[X][i]
# 		Y = data['Y'][i] 
# 		sequence = [onehot(vocab_lst.index(s), len(vocab_lst)) for s in antigen] 
# 		labels = [0 for i in range(len(antigen))]
# 		mask = [True for i in range(len(labels))] # labels and mask have the same length
# # 		sequence += (maxlength-len(sequence)) * [zerohot(len(vocab_lst))] #pad to consistent length
# # 		labels += (maxlength-len(labels)) * [0] 
# # 		mask += (maxlength-len(mask)) * [False] # pad to maxlength
# 		for y in Y:
# 			labels[y] = 1 		
# # 		sequence, labels, mask = sequence[:maxlength], labels[:maxlength], mask[:maxlength]
# 		sequence, labels, mask = torch.FloatTensor(sequence), torch.FloatTensor(labels), torch.BoolTensor(mask) 
# 		# print(sequence.shape, labels.shape, mask.shape)
#         # sequence is 2D, labels and mask are 1D
# 		standard_data.append((sequence, labels, mask))
# 	return standard_data 

In [87]:
# train_data_stand2 = standardize_data2(train_data, vocab_lst)
# valid_data_stand2 = standardize_data2(valid_data, vocab_lst)
# test_data_stand2 = standardize_data2(test_data, vocab_lst)

In [88]:
train_data_stand = standardize_data(train_data, vocab_lst)
valid_data_stand = standardize_data(valid_data, vocab_lst)
test_data_stand = standardize_data(test_data, vocab_lst)

In [90]:
class dataset(Dataset):
	def __init__(self, data):
		self.sequences = [i[0] for i in data]
		self.labels = [i[1] for i in data]
		self.mask = [i[2] for i in data] 

	def __getitem__(self, index):
		return self.sequences[index], self.labels[index], self.mask[index]

	def __len__(self):
		return len(self.labels)

In [91]:
train_set = dataset(train_data_stand)
valid_set = dataset(valid_data_stand)
test_set = dataset(test_data_stand)
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False)

In [110]:
for sequence, labels, mask in tqdm(train_loader):
    
    print(sequence.shape, labels.shape, mask.shape)
# for batch, (x,y,z) in enumerate(train_loader):
#         print(batch)

100%|██████████| 139/139 [00:00<00:00, 1440.68it/s]

torch.Size([16, 300, 24]) torch.Size([16, 300]) torch.Size([16, 300])
torch.Size([16, 300, 24]) torch.Size([16, 300]) torch.Size([16, 300])
torch.Size([16, 300, 24]) torch.Size([16, 300]) torch.Size([16, 300])
torch.Size([16, 300, 24]) torch.Size([16, 300]) torch.Size([16, 300])
torch.Size([16, 300, 24]) torch.Size([16, 300]) torch.Size([16, 300])
torch.Size([16, 300, 24]) torch.Size([16, 300]) torch.Size([16, 300])
torch.Size([16, 300, 24]) torch.Size([16, 300]) torch.Size([16, 300])
torch.Size([16, 300, 24]) torch.Size([16, 300]) torch.Size([16, 300])
torch.Size([16, 300, 24]) torch.Size([16, 300]) torch.Size([16, 300])
torch.Size([16, 300, 24]) torch.Size([16, 300]) torch.Size([16, 300])
torch.Size([16, 300, 24]) torch.Size([16, 300]) torch.Size([16, 300])
torch.Size([16, 300, 24]) torch.Size([16, 300]) torch.Size([16, 300])
torch.Size([16, 300, 24]) torch.Size([16, 300]) torch.Size([16, 300])
torch.Size([16, 300, 24]) torch.Size([16, 300]) torch.Size([16, 300])
torch.Size([16, 300,

In [104]:
embedding = nn.Embedding(10, 3)
i = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
o = embedding(i)
i.shape, o.shape

(torch.Size([2, 4]), torch.Size([2, 4, 3]))

In [51]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

In [21]:
# train_data_stand[0][2].shape

torch.Size([300])

In [28]:
def plot(label_lst, predict_lst, name):

    fpr, tpr, thresholds = metrics.roc_curve(label_lst, predict_lst, )
    roc_auc = auc(fpr, tpr)

    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.savefig(name)

In [255]:
loss = nn.BCEWithLogitsLoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(input, target)
input, target, output, output.backward()

(tensor([-0.2493,  0.2576, -0.0780], requires_grad=True),
 tensor([1., 0., 0.]),
 tensor(0.7702, grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 None)

In [256]:
n[0] + output

tensor(-0.5674, grad_fn=<AddBackward0>)

In [241]:
output = torch.FloatTensor([output.item()])
c, c.shape, output, output.shape

(tensor([[-1.1693, -1.0862,  0.2509, -2.3874]]),
 torch.Size([1, 4]),
 tensor([1.2722]),
 torch.Size([1]))

In [245]:
output = torch.unsqueeze(output, 0)
output.shape

torch.Size([1, 1])

In [251]:

torch.cat((c,output), dim=1)

tensor([[-1.1693, -1.0862,  0.2509, -2.3874,  1.2722]])

In [168]:
m = Categorical(torch.tensor([ 0.25, 0.25, 0.25, 0.25]))
m.sample()

tensor(0)

In [208]:
b = torch.randn(4, 1)
c = torch.randn(1, 4)

In [209]:
b, b.shape, c, c.shape

(tensor([[-0.3045],
         [-0.1741],
         [ 0.1884],
         [-1.2225]]),
 torch.Size([4, 1]),
 tensor([[-1.1693, -1.0862,  0.2509, -2.3874]]),
 torch.Size([1, 4]))

In [210]:
loss = torch.mul(b, c).mul(-1)
m = torch.sum(loss, dim=-1)
loss, m

(tensor([[-0.3561, -0.3308,  0.0764, -0.7271],
         [-0.2035, -0.1891,  0.0437, -0.4156],
         [ 0.2203,  0.2046, -0.0473,  0.4498],
         [-1.4295, -1.3279,  0.3067, -2.9186]]),
 tensor([-1.3376, -0.7645,  0.8275, -5.3694]))

In [254]:
n = torch.sum(torch.mul(b, c).mul(-1), -1)
n[0]

tensor(-1.3376)

In [196]:
b = torch.tensor([[0, 1], [2, 3]])
b2 = torch.reshape(b, (-1,))
b3 = torch.unsqueeze(b2, 1)
b, b2, b2.shape, b3, b3.shape

(tensor([[0, 1],
         [2, 3]]), tensor([0, 1, 2, 3]), torch.Size([4]), tensor([[0],
         [1],
         [2],
         [3]]), torch.Size([4, 1]))

In [257]:
rewards = []
rewards.insert(0,2)
rewards.insert(0,3)
rewards

[3, 2]

In [216]:
a = torch.randn(5,1)
torch.log(a)

tensor([[-0.8064],
        [-2.4553],
        [    nan],
        [    nan],
        [-0.1831]])

In [206]:
class RNN(nn.Module):
    def __init__(self, name, hidden_size, input_size, num_layers = 2):
        super(RNN, self).__init__()
        self.name = name 
        self.hidden_size = hidden_size
        self.input_size = input_size 
        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=input_size,
            hidden_size=hidden_size,         # rnn hidden unit
            num_layers=num_layers,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.out = nn.Linear(hidden_size, 1)
        criterion = torch.nn.BCEWithLogitsLoss()  
        self.opt = torch.optim.Adam(self.parameters(), lr=1e-3)
    
    
    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out)
        out = out.squeeze(-1)
        return out
    
    def learn(self, sequence, labels, mask):
        prediction_lst = []
        label_lst = []
        prediction = self.forward(sequence)
        pred = torch.sigmoid(prediction)
        print(prediction, prediction.shape)
#         print(pred, pred.shape)
        print(labels)
#         print(mask)
        for pred, label, msk in zip(prediction, labels, mask):
                num = sum(msk.tolist()) 
                pred = pred.tolist()[:num] 
                label = label.tolist()[:num] 
                label_lst.extend(label)
                prediction_lst.extend(pred)
#         print(prediction_lst)
        sort_pred = deepcopy(prediction_lst)
        sort_pred.sort() 
        threshold = sort_pred[int(len(sort_pred)*0.9)]
#         print(threshold, type(threshold))
#         print(type(prediction_lst))
#         threshold = 0.5
        float2binary = lambda x:0 if x<threshold else 1
        binary_pred_lst = list(map(float2binary, prediction_lst))
#         print(binary_pred_lst, len(binary_pred_lst))
        
#         print("size", prediction.shape, labels.shape, mask.shape)
        criterion = torch.nn.BCEWithLogitsLoss(size_average=True, weight = mask)  
        
        loss = criterion(prediction, labels)
        print(loss)
        self.opt.zero_grad() 
        loss.backward() 
        self.opt.step()
        
    def test(self, test_loader, name):
        label_lst, prediction_lst = [], []
        for sequence, labels, mask in test_loader:
            prediction = self.forward(sequence)
            prediction = torch.sigmoid(prediction)
#             print(prediction)
            for pred, label, msk in zip(prediction, labels, mask):
                num = sum(msk.tolist()) 
                pred = pred.tolist()[:num] 
                label = label.tolist()[:num] 
                label_lst.extend(label)
                prediction_lst.extend(pred)
#         print(prediction_lst)
        sort_pred = deepcopy(prediction_lst)
        sort_pred.sort() 
        threshold = sort_pred[int(len(sort_pred)*0.9)]
        print(threshold, type(threshold))
        print(type(prediction_lst))
        float2binary = lambda x:0 if x<threshold else 1
        binary_pred_lst = list(map(float2binary, prediction_lst))
        plot(label_lst, prediction_lst, name)
        print('roc_auc', roc_auc_score(label_lst, prediction_lst), 
    		  'F1', f1_score(label_lst, binary_pred_lst), 
    		  'prauc', average_precision_score(label_lst, binary_pred_lst))
        

In [207]:
model = RNN(name = 'Epitope', hidden_size=100, input_size=len(vocab_lst))
epoch = 10
for ep in range(epoch):
    for sequence, labels, mask in train_loader:
#         print(sequence.shape, labels.shape, mask.shape)
        model.learn(sequence, labels, mask)
    model.test(test_loader, name = model.name + '_' + str(ep) + '.png')

tensor([[-0.0895, -0.0885, -0.0876,  ..., -0.0886, -0.0893, -0.0923],
        [-0.0895, -0.0917, -0.0910,  ..., -0.0900, -0.0900, -0.0900],
        [-0.0895, -0.0885, -0.0876,  ..., -0.0869, -0.0874, -0.0879],
        ...,
        [-0.0895, -0.0902, -0.0912,  ..., -0.0900, -0.0900, -0.0900],
        [-0.0890, -0.0885, -0.0879,  ..., -0.0900, -0.0900, -0.0900],
        [-0.0895, -0.0902, -0.0899,  ..., -0.0954, -0.0959, -0.0978]],
       grad_fn=<SqueezeBackward1>) torch.Size([16, 300])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor(0.5078, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor([[-0.0976, -0.1075, -0.1146,  ..., -0.1255, -0.1253, -0.1244],
        [-0.0962, -0.1056, -0.1129,  ..., -0.1216, -0.1216, -0.1216],
        [-0.0971, -0.1025, -0.1073,  ..., -0.1216, -

tensor([[-0.1789, -0.2734, -0.3669,  ..., -0.9335, -0.9386, -0.9426],
        [-0.1789, -0.2757, -0.3712,  ..., -0.9397, -0.9425, -0.9461],
        [-0.1789, -0.2766, -0.3704,  ..., -0.9798, -0.9744, -0.9685],
        ...,
        [-0.1789, -0.2779, -0.3809,  ..., -0.9250, -0.9238, -0.9287],
        [-0.1789, -0.2740, -0.3702,  ..., -0.9332, -0.9375, -0.9419],
        [-0.1789, -0.2734, -0.3658,  ..., -0.9475, -0.9462, -0.9474]],
       grad_fn=<SqueezeBackward1>) torch.Size([16, 300])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor(0.3254, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor([[-0.1883, -0.2967, -0.4120,  ..., -1.2728, -1.2718, -1.2697],
        [-0.1891, -0.2994, -0.4130,  ..., -1.2938, -1.2921, -1.2903],
        [-0.1883, -0.3009, -0.4178,  ..., -1.2751, -

KeyboardInterrupt: 